# Version 2.4: 웨이팅 핫플레이스 요약문 생성

이 노트북은 **리뷰 데이터를 사용하지 않고**, 외부 정보만으로 웨이팅 핫플레이스의 요약문을 생성합니다.

## 📌 핵심 특징

### 처리 모드
- 🔹 **단일 매장 모드**: 한 매장씩 상세하게 처리
- 🔹 **다중 매장 모드**: 여러 매장을 일괄 처리 (배치)

### 데이터 소스
- 🔥 **웨이팅 정보** (평균 대기시간, 피크 타임)
- 📸 **SNS/비주얼** (인스타그래머블, 포토존)
- 🎨 **독특한 컨셉** (차별화 요소, 특이한 메뉴)
- 🌊 **트렌드/화제성** (바이럴, 미디어 노출)
- ⏰ **대기 가치** (특별한 경험, 웨이팅 이유)
- 🎯 **타겟 고객** (MZ세대, 20-30대)

### 프롬프트 아키텍처
- **시스템/유저 프롬프트 분리 패턴** 사용
- **Few-shot Learning**: 10개의 고품질 예시로 학습
- **JSON 출력 강제**: 구조화된 결과 보장

### 출력 형식
```json
{
  "shop_seq": 1,
  "shop_name": "매장명",
  "title": "SNS 핫플 성수동에서 크림 폭포 크로플로 감탄하는 디저트 카페",
  "summaries": [
    "화제성 + 웨이팅 정보 문장 (35-55자)",
    "시그니처 메뉴 비주얼 + 경험 문장 (35-55자)",
    "공간 감성 + 포토존 + 대기 가치 문장 (35-55자)"
  ]
}
```

## 🚀 사용 워크플로우

### 단일 매장 모드
```
1. 섹션 1-3: 환경 설정 (최초 1회)
2. 섹션 4: MODE = "single" 설정 + 매장명 입력
3. 섹션 5: 웹 검색 정보 수집 및 붙여넣기
4. 섹션 6: 프롬프트 실행 → 요약문 생성
5. 섹션 7: 결과 검증 (선택)
6. 섹션 8: JSON 파일 저장 (선택)
```

### 다중 매장 모드
```
1. 섹션 1-3: 환경 설정 (최초 1회)
2. 섹션 4: MODE = "multi" 설정 + SHOP_LIST에 매장 정보 입력
3. 섹션 5: 건너뛰기
4. 섹션 6: 프롬프트 실행 → 여러 매장 일괄 생성
5. 섹션 7: 결과 검증 (선택)
6. 섹션 8: JSON + CSV 파일 저장 (선택)
```

## ⚠️ 섹션 1-3: 최초 1회만 실행 (수정 금지)

In [ ]:
# 섹션 1: Package 설치
!pip install google-cloud-aiplatform google-genai python-dotenv -q

In [ ]:
# 섹션 2: 라이브러리 Import
import os
import json
import pandas as pd
import vertexai
from google import genai
from google.genai import types
from dotenv import load_dotenv

In [ ]:
# 섹션 3: Vertex AI 초기화
# 환경 변수 로드
load_dotenv()

# GCP 프로젝트 설정
PROJECT_ID = "wad-dw"  # 실제 GCP 프로젝트 ID (프로젝트 이름 아님!)
LOCATION = "us-central1"

# Vertex AI 초기화
vertexai.init(project=PROJECT_ID, location=LOCATION)
client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

print(f"✅ Vertex AI 초기화 완료: {PROJECT_ID} ({LOCATION})")

## ✅ 섹션 4: 매장 정보 입력 (수정 가능)

### 처리 모드 선택

#### 🔹 단일 매장 모드 (single)
- 한 매장씩 상세하게 처리
- 섹션 5에서 정보 수집
- 결과: JSON 파일 1개

#### 🔹 다중 매장 모드 (multi)
- 여러 매장을 일괄 처리
- SHOP_LIST에 모든 매장 정보 입력
- 섹션 5 건너뛰기
- 결과: JSON + CSV 파일

In [ ]:
# ========================================
# 매장 정보 입력
# ========================================
# 
# 두 가지 모드 지원:
# 1. 단일 매장 모드: SHOP_NAME에 하나의 매장명 입력
# 2. 다중 매장 모드: SHOPS 리스트 + COLLECTED_INFO 딕셔너리로 관리
# ========================================

# -----------------------------------------
# 모드 선택
# -----------------------------------------
MODE = "multi"  # "single" 또는 "multi"

# -----------------------------------------
# [단일 매장 모드] 단일 매장 정보
# -----------------------------------------
if MODE == "single":
    SHOP_SEQ = 1
    SHOP_NAME = "매장명"
    
    print(f"📍 단일 매장 모드")
    print(f"   - 매장명: {SHOP_NAME}")
    print(f"   - Shop Seq: {SHOP_SEQ}")

# -----------------------------------------
# [다중 매장 모드] 여러 매장 정보
# -----------------------------------------
elif MODE == "multi":
    # 1️⃣ 매장 기본 정보 (shop_seq, shop_name)
    SHOPS = [
        (1, "달맞이 광장 바베큐"),
        (2, "이재모피자 (본점)"),
        (3, "자매국수"),
        (4, "하이디라오 (명동점)"),
        (5, "산청숯불가든 (을지로)"),
        (6, "숙성도 (중문점)"),
        (7, "톤쇼우 (광안점)"),
        (8, "연돈"),
        (9, "카와카츠 (합정점)"),
        (10, "고기리막국수"),
    ]
    
    # 2️⃣ 매장별 수집된 정보 (매장명을 키로 사용)
    COLLECTED_INFO = {
        "달맞이 광장 바베큐": "",
        "이재모피자 (본점)": "",
        "자매국수": "",
        "하이디라오 (명동점)": "",
        "산청숯불가든 (을지로)": "",
        "숙성도 (중문점)": "",
        "톤쇼우 (광안점)": "",
        "연돈": "",
        "카와카츠 (합정점)": "",
        "고기리막국수": "",
    }
    
    # 3️⃣ SHOP_LIST 자동 생성 (shop_seq, shop_name, collected_info 매핑)
    SHOP_LIST = [
        {
            "shop_seq": seq,
            "shop_name": name,
            "collected_info": COLLECTED_INFO.get(name, "[정보 없음 - COLLECTED_INFO에 추가 필요]")
        }
        for seq, name in SHOPS
    ]
    
    # 출력
    print(f"📍 다중 매장 모드")
    print(f"   - 총 매장 수: {len(SHOP_LIST)}개")
    for idx, shop in enumerate(SHOP_LIST, 1):
        info_length = len(shop['collected_info'].strip())
        status = "✅" if info_length > 100 else "⚠️"
        print(f"   {status} {idx}. {shop['shop_name']} (seq: {shop['shop_seq']}) - {info_length:,}자")
    print(f"\n⚠️ 섹션 5는 건너뛰고 바로 섹션 6을 실행하세요")

## ✅ 섹션 5: 웹 검색 및 정보 수집

### 정보 수집 방법

웨이팅 핫플레이스는 다음 정보를 중심으로 수집합니다:

- 🔥 **웨이팅 정보** (평균 대기시간: 30분/1시간/2시간, 피크 타임)
- 📸 **SNS/비주얼** (인스타그래머블 포인트, 대표 메뉴 사진)
- 🎨 **독특한 컨셉** (특이한 메뉴 조합, 비주얼 임팩트)
- 🌊 **트렌드/화제성** (SNS 바이럴, 연예인 방문, 미디어 노출)
- ⏰ **대기 가치** (웨이팅해도 갈 만한 이유, 특별한 경험)
- 🎯 **타겟 고객** (MZ세대, 20-30대 여성, 커플)
- 📍 **지역/감성** (성수동 힙함, 한남동 세련됨)

### 수집 프로세스

1. 매장명으로 SNS 검색 (Instagram, 네이버 블로그)
2. 위 정보를 중심으로 수집
3. 아래 `collected_info` 변수에 붙여넣기
4. 섹션 6 실행하여 요약문 생성

In [ ]:
# 📝 수동 입력: 웹에서 수집한 정보를 아래에 입력하세요
# (단일 매장 모드에서만 사용)

if MODE == "single":
    collected_info = """
    [여기에 웹 검색 결과를 붙여넣으세요]

    예시:
    - 웨이팅: 주말 2시간, 평일 1시간
    - 화제성: SNS에서 크림 폭포 크로플로 바이럴
    - 시그니처: 크림 크로플 (높이 20cm), 딸기 케이크 산
    - 비주얼: 생크림이 폭포처럼 흘러내림
    - 타겟: MZ세대, 20-30대 여성
    - 공간: 화이트톤 미니멀, 자연광 통창
    - 포토존: 창가 자리, 대형 통창
    - 경험: 인스타그램 감성, 브런치
    """
    
    print("📥 웹 검색 정보 수집 완료 (단일 매장 모드)")
    print(f"   - 수집된 정보 길이: {len(collected_info):,} 글자")
    print(f"   - 추천 길이: 300~1,000 글자")
    if len(collected_info) < 200:
        print("   ⚠️ 정보가 너무 적습니다. 더 많은 정보를 수집해주세요.")

elif MODE == "multi":
    print("📥 다중 매장 모드")
    print("   ⚠️ 이 섹션은 건너뛰세요. 섹션 4에서 이미 정보를 입력했습니다.")
    print("   ➡️ 바로 섹션 6을 실행하세요.")

## ✅ 섹션 6: 프롬프트 작성 및 요약문 생성 (수정 가능)

### 프롬프트 구조 설계

이 노트북은 **시스템/유저 프롬프트 분리 패턴**을 사용합니다:

#### 📌 시스템 프롬프트 (System Prompt)
- **역할**: AI의 정체성, 규칙, 출력 형식 정의
- **특징**: 고정된 내용 (매장마다 동일)
- **내용**: 웨이팅 핫플 작성 원칙, 구조 규칙, 출력 형식

#### 📌 유저 프롬프트 (User Prompt)
- **역할**: 구체적인 작업 요청 및 데이터 제공
- **특징**: 매장마다 변경되는 내용
- **내용**: 매장 정보, 수집된 정보, Few-shot 예시

#### 🔗 프롬프트 결합
Google Gemini는 시스템/유저를 API 레벨에서 구분하지 않으므로, 전송 시 하나의 문자열로 결합합니다.

### 프롬프트 작성 규칙
- ❌ **f-string 사용 금지** (변수가 아직 정의되지 않아 에러)
- ✅ **일반 문자열 + `.format()` 사용**
- ✅ **JSON 예시에서 `{{` `}}` 사용** (이스케이프)

In [ ]:
# ========================================
# 프롬프트 정의
# ========================================

# -----------------------------------------
# [시스템 프롬프트] AI의 정체성과 규칙
# -----------------------------------------
system_prompt = """
당신은 웨이팅 핫플레이스 전문 작가입니다.
리뷰 데이터 없이 외부 정보만으로 "왜 줄 서서라도 가야 하는가?"를 명확히 전달하는 매장 요약문을 작성합니다.

## 작성 원칙
1. **화제성**: SNS 바이럴, 입소문, 미디어 노출을 강조
2. **비주얼**: 인스타그래머블, 포토제닉한 요소 부각
3. **경험 가치**: 웨이팅해도 갈 만한 특별한 경험 전달
4. **생동감**: 흥미롭고 트렌디한 어조 사용
5. **대기 정당화**: 웨이팅 시간과 대기 가치를 명확히 제시
6. **독창성**: 예시를 그대로 복사하지 말고, 톤앤매너만 참고하여 해당 매장만의 고유한 특징을 작성

## 요약 문장 구조
- **1번 문장**: 지역/컨셉 + 화제성 키워드 + 웨이팅 정보 + 타겟 고객층
- **2번 문장**: 시그니처 메뉴 비주얼 + 독특한 경험 + 맛/식감
- **3번 문장**: 공간 감성 + 포토존 + 방문 용도 + 대기 가치

## 제목 작성 규칙
- 형식: "[화제성 키워드]에서, [시그니처 비주얼 메뉴]로 [감정 동사]하는 [컨셉]"
- 예시: "SNS 핫플 성수동에서, 크림 폭포 크로플로 감탄하는 디저트 카페"
- 길이: 20-40자
- 화제성 키워드 필수 포함

## 웨이팅 핫플 특화 가이드
- ✅ 웨이팅 정보 명시 (주말 2시간, 평일 1시간 등)
- ✅ 화제성 강조 (SNS 바이럴, 입소문, 연예인 단골)
- ✅ 비주얼 임팩트 (높이 쌓인, 폭포처럼, 불쇼)
- ✅ 타겟 고객층 (MZ세대, 20-30대 여성, 커플)
- ✅ 감성 표현 (감탄, 놀라움, 매료, 황홀)
- ✅ 포토존 언급 (인스타 성지, 창가 자리)
- ✅ 경험 가치 (색다른, 특별한, 이색적)
- ❌ 가성비/실용성 표현 금지
- ❌ 조용한/차분한 표현 금지

## 출력 형식
반드시 유효한 JSON 형식으로만 출력하세요. 다른 설명이나 주석은 포함하지 마세요.
{{
  "shop_seq": 숫자,
  "shop_name": "매장명",
  "title": "제목",
  "summaries": ["문장1", "문장2", "문장3"]
}}
"""

# -----------------------------------------
# [유저 프롬프트 템플릿] 구체적 작업 내용
# -----------------------------------------
user_prompt_template = """
[매장 정보]
매장명: {shop_name}
Shop Seq: {shop_seq}

[수집된 정보]
{collected_info}

[참고 예시 - 아래 예시들의 톤앤매너와 문체만 참고하세요. 내용은 절대 복사하지 마세요.]

**중요**: 아래 예시는 작성 스타일을 보여주기 위한 참고용입니다. 예시의 내용을 그대로 사용하지 말고, 
수집된 정보를 바탕으로 해당 매장만의 고유한 특징을 작성하세요.

---

**예시 1: 디저트 카페 (성수동)**
{{
  "title": "SNS 핫플 성수동에서, 크림 폭포 크로플로 감탄하는 디저트 카페",
  "summaries": [
    "성수동에서 크림이 폭포처럼 쏟아지는 크로플을 내세워 MZ세대 사이에서 SNS 바이럴로 주말 2시간 웨이팅을 기록하는 핫플레이스입니다.",
    "시그니처인 크림 크로플은 높이 20cm로 쌓인 생크림이 폭포처럼 흘러내리는 비주얼로 인스타그램을 점령하며, 바삭한 와플과 부드러운 크림의 조화가 중독성을 선사합니다.",
    "화이트톤 미니멀 인테리어와 대형 통창이 어우러진 공간은 자연광이 쏟아지는 창가 자리가 인기 포토존으로, 친구와의 브런치나 데이트 장소로 SNS 감성을 완성합니다."
  ]
}}

**예시 2: 일식당 (망원동)**
{{
  "title": "웨이팅 필수 망원동, 숯불 직화 연어덮밥으로 매료되는 일식당",
  "summaries": [
    "망원동에서 숯불 직화로 구운 연어덮밥을 내세워 20-30대 여성 사이에서 입소문으로 평일에도 1시간 웨이팅이 필수인 일식당입니다.",
    "시그니처인 숯불 연어덮밥은 눈앞에서 직화로 그을리는 퍼포먼스가 압권이며, 겉은 고소하고 속은 부드러운 연어의 식감이 감탄을 자아냅니다.",
    "빈티지 일본 선술집 감성의 아늑한 공간은 오픈 키친에서 펼쳐지는 조리 과정을 생생히 볼 수 있어, 색다른 일식 경험을 원하는 이들에게 긴 웨이팅을 감수할 만한 가치를 제공합니다."
  ]
}}

**예시 3: 화덕 피자 (이태원)**
{{
  "title": "입소문 난 이태원, 화덕 직화 나폴리 피자로 황홀해지는 이탈리안",
  "summaries": [
    "이태원에서 화덕 직화로 구운 정통 나폴리 피자를 내세워 미식가들 사이에서 입소문으로 오픈 시간부터 웨이팅 줄이 늘어서는 핫플레이스입니다.",
    "시그니처인 마르게리타는 화덕에서 60초 만에 구워져 가장자리가 표범 무늬처럼 그을린 비주얼이 압권이며, 쫄깃한 도우와 모짜렐라의 풍미가 입안에서 황홀한 조화를 이룹니다.",
    "오픈 키친의 화덕을 중심으로 설계된 공간은 피자가 만들어지는 과정을 실시간으로 감상할 수 있어, 긴 대기 시간을 감수할 만한 특별한 경험을 선사합니다."
  ]
}}

**예시 4: 국수집 (종로)**
{{
  "title": "화제의 종로 골목, 손칼국수 한 그릇으로 설레는 전통 맛집",
  "summaries": [
    "종로 골목에서 할머니가 직접 밀어내는 손칼국수를 내세워 전 연령층 사이에서 입소문으로 점심시간 30분 이상 웨이팅이 기본인 전통 맛집입니다.",
    "시그니처인 손칼국수는 쫄깃한 면발과 진한 육수가 어우러져 소박하지만 깊은 맛을 구현하며, 수십 년 노하우가 담긴 김치와 함께 먹는 조합이 중독적입니다.",
    "옛 정취가 느껴지는 레트로 감성의 공간은 오래된 테이블과 의자가 향수를 불러일으키며, 할머니의 손맛을 경험하려는 이들에게 줄 서서라도 갈 가치를 제공합니다."
  ]
}}

**예시 5: 중식당 (명동)**
{{
  "title": "SNS 화제 명동, 마라탕 퍼포먼스로 놀라는 중국 핫팟",
  "summaries": [
    "명동에서 직원이 직접 면을 뽑아주는 퍼포먼스를 내세워 외국인 관광객과 MZ세대 사이에서 SNS 바이럴로 주말 1시간 이상 웨이팅이 필수인 중식당입니다.",
    "시그니처인 마라탕은 테이블 앞에서 면을 뽑는 쇼맨십이 압권이며, 얼얼한 마라 향과 쫄깃한 수타면의 조합이 중독적인 풍미를 선사합니다.",
    "화려한 중국 전통 장식과 붉은 조명이 어우러진 공간은 이국적인 분위기를 자아내며, 퍼포먼스와 비주얼을 모두 즐기려는 이들에게 긴 웨이팅을 감수할 만한 가치를 제공합니다."
  ]
}}

**예시 6: 고기집 (을지로)**
{{
  "title": "을지로 핫플, 숯불 직화 LA갈비로 감동하는 고깃집",
  "summaries": [
    "을지로에서 숯불 직화로 구운 LA갈비를 내세워 직장인과 젊은 층 사이에서 입소문으로 저녁 시간 1시간 웨이팅이 기본인 고깃집입니다.",
    "시그니처인 LA갈비는 숯불에 직화로 구워져 겉은 바삭하고 속은 촉촉한 식감을 자랑하며, 달콤한 양념이 고기 육즙과 어우러져 감탄을 자아냅니다.",
    "레트로 감성의 을지로 무드를 살린 공간은 노포의 향수와 힙한 분위기가 공존하며, 특별한 날 고기를 즐기려는 이들에게 웨이팅을 감수할 만한 가치를 제공합니다."
  ]
}}

**예시 7: 고기 뷔페 (제주)**
{{
  "title": "제주 화제 맛집, 흑돼지 무한리필로 만족하는 고기 뷔페",
  "summaries": [
    "제주 중문에서 흑돼지 무한리필을 내세워 관광객과 현지인 사이에서 입소문으로 점심 저녁 모두 30분 이상 웨이팅이 필수인 고기 뷔페입니다.",
    "시그니처인 제주 흑돼지는 구이용, 찌개용 등 다양한 부위를 무제한으로 즐길 수 있으며, 육즙이 풍부하고 고소한 맛이 일품입니다.",
    "넓고 쾌적한 공간은 가족 단위 방문객에게 최적화되어 있으며, 무한리필이라는 가치와 흑돼지 품질로 긴 대기를 감수할 만한 만족도를 제공합니다."
  ]
}}

**예시 8: 돈카츠 (부산)**
{{
  "title": "부산 광안리, 바삭한 히레카츠로 황홀해지는 돈카츠 맛집",
  "summaries": [
    "부산 광안리에서 바삭한 히레카츠를 내세워 부산 현지인과 관광객 사이에서 입소문으로 주말 1시간 웨이팅이 기본인 돈카츠 전문점입니다.",
    "시그니처인 히레카츠는 겉은 황금빛 바삭함을 자랑하고 속은 부드럽고 육즙이 풍부한 안심 부위로, 한 입 베어 물면 육즙이 터지는 식감이 감탄을 자아냅니다.",
    "바다가 보이는 광안리 뷰를 갖춘 공간은 통창으로 광안대교를 감상하며 식사할 수 있어, 특별한 경험을 원하는 이들에게 웨이팅을 감수할 만한 가치를 제공합니다."
  ]
}}

**예시 9: 돈카츠 (합정)**
{{
  "title": "합정 핫플, 크림 소스 히레카츠로 감동하는 일식 돈카츠",
  "summaries": [
    "합정에서 크림 소스를 곁들인 히레카츠를 내세워 20-30대 여성 사이에서 SNS 바이럴로 평일에도 40분 웨이팅이 필수인 돈카츠 전문점입니다.",
    "시그니처인 크림 히레카츠는 바삭한 튀김옷과 부드러운 안심, 진한 크림 소스의 조합이 압권이며, 독특한 조합이 입안에서 황홀한 풍미를 선사합니다.",
    "일본 감성의 모던한 인테리어는 깔끔하고 아늑한 분위기를 자아내며, 색다른 돈카츠 경험을 원하는 이들에게 긴 대기를 감수할 만한 만족도를 제공합니다."
  ]
}}

**예시 10: 막국수집 (춘천)**
{{
  "title": "춘천 핫플, 메밀향 가득 막국수로 시원해지는 전통 맛집",
  "summaries": [
    "춘천에서 직접 뽑은 메밀 막국수를 내세워 현지인과 관광객 사이에서 입소문으로 여름철 1시간 이상 웨이팅이 필수인 전통 맛집입니다.",
    "시그니처인 메밀 막국수는 쫄깃한 면발과 시원한 육수가 어우러져 더운 여름 갈증을 해소하며, 직접 담근 겨자와 함께 먹는 조합이 중독적입니다.",
    "시골 한옥을 개조한 공간은 정원이 내려다보이는 평상 자리가 인기이며, 전통 막국수의 깊은 맛을 경험하려는 이들에게 줄 서서라도 갈 가치를 제공합니다."
  ]
}}

---

**다시 한 번 강조**: 위 예시는 작성 스타일(톤, 문체, 구조)을 참고하기 위한 것입니다.
예시의 구체적인 내용(메뉴명, 지역명, 특징 등)을 그대로 사용하지 말고,
[수집된 정보]를 바탕으로 [{shop_name}] 매장만의 고유한 특징을 작성하세요.
"""

print("✅ 프롬프트 템플릿 작성 완료")
print("   - 시스템 프롬프트: 웨이팅 핫플 작성 원칙")
print("   - 유저 프롬프트: 구체적 작업 내용")
print("   - Few-shot 예시: 10개 (톤앤매너 참고용)")

In [ ]:
# ========================================
# 모델 설정 및 LLM 호출
# ========================================

# -----------------------------------------
# 모델 파라미터 설정
# -----------------------------------------
MODEL_NAME = "gemini-2.5-pro"  # 높은 품질
TEMPERATURE = 0.6  # 중저가보다 약간 높은 창의성
MAX_OUTPUT_TOKENS = 4096  # JSON 응답을 위한 충분한 토큰

generation_config = types.GenerateContentConfig(
    temperature=TEMPERATURE,
    max_output_tokens=MAX_OUTPUT_TOKENS,
    response_mime_type="application/json"  # JSON 출력 강제
)

print(f"🤖 모델: {MODEL_NAME}")
print(f"   - Temperature: {TEMPERATURE}")
print(f"   - Max Output Tokens: {MAX_OUTPUT_TOKENS:,}\n")

# -----------------------------------------
# 요약문 생성 함수
# -----------------------------------------
def generate_summary(shop_seq, shop_name, collected_info):
    """단일 매장에 대한 요약문 생성"""
    
    # 유저 프롬프트 생성
    user_prompt = user_prompt_template.format(
        shop_seq=shop_seq,
        shop_name=shop_name,
        collected_info=collected_info
    )
    
    # 프롬프트 결합
    combined_prompt = f"{system_prompt}\n\n{user_prompt}"
    
    try:
        # LLM 호출
        response = client.models.generate_content(
            model=MODEL_NAME,
            contents=combined_prompt,
            config=generation_config
        )
        
        # JSON 파싱
        result_text = response.text
        result_json = json.loads(result_text)
        
        # 토큰 사용량 추출
        usage_metadata = response.usage_metadata if hasattr(response, 'usage_metadata') else None
        token_info = {}
        if usage_metadata:
            token_info = {
                "input_tokens": usage_metadata.prompt_token_count,
                "output_tokens": usage_metadata.candidates_token_count,
                "total_tokens": usage_metadata.total_token_count
            }
        
        return {
            "status": "success",
            "data": result_json,
            "tokens": token_info
        }
        
    except json.JSONDecodeError as e:
        return {
            "status": "error",
            "error_type": "JSONDecodeError",
            "message": str(e),
            "response": response.text if 'response' in locals() else None,
            "tokens": {}
        }
        
    except Exception as e:
        return {
            "status": "error",
            "error_type": type(e).__name__,
            "message": str(e),
            "response": response.text if 'response' in locals() else None,
            "tokens": {}
        }

# -----------------------------------------
# 단일/다중 모드 처리
# -----------------------------------------
all_results = []
total_input_tokens = 0
total_output_tokens = 0
total_tokens = 0

if MODE == "single":
    # 단일 매장 모드
    print(f"📝 [{SHOP_NAME}] 요약문 생성 중...\n")
    
    result = generate_summary(SHOP_SEQ, SHOP_NAME, collected_info)
    
    if result["status"] == "success":
        result_json = result["data"]
        all_results.append(result_json)
        
        # 토큰 사용량 출력
        tokens = result.get("tokens", {})
        if tokens:
            print(f"📊 토큰 사용량:")
            print(f"   - Input:  {tokens.get('input_tokens', 0):,} tokens")
            print(f"   - Output: {tokens.get('output_tokens', 0):,} tokens")
            print(f"   - Total:  {tokens.get('total_tokens', 0):,} tokens\n")
        
        print("✅ 요약문 생성 완료\n")
        print("=" * 80)
        print(json.dumps(result_json, ensure_ascii=False, indent=2))
        print("=" * 80)
    else:
        print(f"❌ 에러 발생: {result['error_type']}")
        print(f"   메시지: {result['message']}")
        if result.get('response'):
            print(f"   응답: {result['response'][:200]}...")

elif MODE == "multi":
    # 다중 매장 모드
    print(f"📝 {len(SHOP_LIST)}개 매장 요약문 생성 시작\n")
    print("=" * 80)
    
    for idx, shop in enumerate(SHOP_LIST, 1):
        shop_seq = shop["shop_seq"]
        shop_name = shop["shop_name"]
        collected_info = shop["collected_info"]
        
        print(f"\n[{idx}/{len(SHOP_LIST)}] {shop_name} 처리 중...")
        
        result = generate_summary(shop_seq, shop_name, collected_info)
        
        if result["status"] == "success":
            result_json = result["data"]
            all_results.append(result_json)
            
            # 토큰 사용량 출력 및 누적
            tokens = result.get("tokens", {})
            if tokens:
                input_tokens = tokens.get('input_tokens', 0)
                output_tokens = tokens.get('output_tokens', 0)
                total_tokens_shop = tokens.get('total_tokens', 0)
                
                total_input_tokens += input_tokens
                total_output_tokens += output_tokens
                total_tokens += total_tokens_shop
                
                print(f"   ✅ 성공 | Input: {input_tokens:,} | Output: {output_tokens:,} | Total: {total_tokens_shop:,} tokens")
            else:
                print(f"   ✅ 성공")
        else:
            print(f"   ❌ 실패: {result['error_type']} - {result['message']}")
            # 실패한 경우에도 기록
            all_results.append({
                "shop_seq": shop_seq,
                "shop_name": shop_name,
                "status": "error",
                "error": result['message']
            })
    
    print("\n" + "=" * 80)
    print(f"\n✅ 전체 완료: {len(all_results)}개 매장 처리")
    
    # 성공/실패 개수
    success_count = sum(1 for r in all_results if r.get("status") != "error")
    error_count = len(all_results) - success_count
    print(f"   - 성공: {success_count}개")
    if error_count > 0:
        print(f"   - 실패: {error_count}개")
    
    # 총 토큰 사용량 출력
    if total_tokens > 0:
        print(f"\n📊 총 토큰 사용량:")
        print(f"   - Total Input:  {total_input_tokens:,} tokens")
        print(f"   - Total Output: {total_output_tokens:,} tokens")
        print(f"   - Total:        {total_tokens:,} tokens")
    
    # -----------------------------------------
    # 전체 결과 상세 출력
    # -----------------------------------------
    print("\n" + "=" * 80)
    print("📋 전체 결과 상세보기")
    print("=" * 80)
    
    for idx, result in enumerate(all_results, 1):
        print(f"\n{'─' * 80}")
        print(f"[{idx}] {result.get('shop_name')}")
        print(f"{'─' * 80}")
        
        if result.get("status") != "error":
            # 성공한 경우 - 전체 내용 출력
            print(f"\n📌 제목:")
            print(f"   {result.get('title')}")
            
            print(f"\n📝 요약문:")
            summaries = result.get('summaries', [])
            for i, summary in enumerate(summaries, 1):
                print(f"   {i}. {summary}")
            
            print(f"\n✅ 상태: 성공")
        else:
            # 실패한 경우
            print(f"\n❌ 상태: 에러")
            print(f"   오류 내용: {result.get('error', 'N/A')}")
    
    print("\n" + "=" * 80)

## ✅ 섹션 7: 후처리 검증 (선택 사항)

### 검증 항목

이 섹션은 생성된 요약문의 품질을 **코드 기반**으로 검증합니다 (LLM 호출 없음):

#### 📋 기본 구조 검증
- 필수 키 존재 확인 (shop_seq, shop_name, title, summaries)
- 요약문 개수 (정확히 3개)
- 문장 길이 (35-55자 권장, 30자 미만/80자 초과 경고)
- 제목 길이 (20-40자 권장)

#### 🚫 품질 검증
- 예시 복사 의심 (예시와 90% 이상 유사)
- 웨이팅 부적합 표현 (가성비, 조용한, 차분한)
- 문장 간 중복 내용 (3개 문장이 너무 유사)
- 특수문자/이모지 체크

#### 💡 검증 결과
- ✅ 통과: 모든 규칙 준수
- ⚠️ 경고: 일부 권장사항 위반 (사용 가능하지만 검토 권장)
- ❌ 실패: 필수 규칙 위반 (수정 필요)

In [ ]:
# ========================================
# 강력한 후처리 검증 함수
# ========================================

import difflib
import re

# -----------------------------------------
# 예시 문장 리스트 (복사 의심 체크용)
# -----------------------------------------
EXAMPLE_SENTENCES = [
    "성수동에서 크림이 폭포처럼 쏟아지는 크로플을 내세워 MZ세대 사이에서 SNS 바이럴로 주말 2시간 웨이팅을 기록하는 핫플레이스입니다.",
    "시그니처인 크림 크로플은 높이 20cm로 쌓인 생크림이 폭포처럼 흘러내리는 비주얼로 인스타그램을 점령하며, 바삭한 와플과 부드러운 크림의 조화가 중독성을 선사합니다.",
    "화이트톤 미니멀 인테리어와 대형 통창이 어우러진 공간은 자연광이 쏟아지는 창가 자리가 인기 포토존으로, 친구와의 브런치나 데이트 장소로 SNS 감성을 완성합니다.",
    "망원동에서 숯불 직화로 구운 연어덮밥을 내세워 20-30대 여성 사이에서 입소문으로 평일에도 1시간 웨이팅이 필수인 일식당입니다.",
    "시그니처인 숯불 연어덮밥은 눈앞에서 직화로 그을리는 퍼포먼스가 압권이며, 겉은 고소하고 속은 부드러운 연어의 식감이 감탄을 자아냅니다.",
]

# -----------------------------------------
# 부적합 키워드 리스트 (웨이팅 핫플에 맞지 않는 표현)
# -----------------------------------------
INAPPROPRIATE_KEYWORDS = [
    "가성비", "합리적", "저렴", "부담 없는",
    "조용한", "차분한", "격식 있는", "고급스러운",
    "실용적", "효율적", "편리한"
]

def calculate_similarity(text1, text2):
    """두 문자열 간 유사도 계산 (0.0 ~ 1.0)"""
    return difflib.SequenceMatcher(None, text1, text2).ratio()

def check_example_similarity(sentence):
    """예시 문장과의 유사도 체크"""
    max_similarity = 0.0
    most_similar_example = None
    
    for example in EXAMPLE_SENTENCES:
        similarity = calculate_similarity(sentence, example)
        if similarity > max_similarity:
            max_similarity = similarity
            most_similar_example = example
    
    return max_similarity, most_similar_example

def check_inappropriate_keywords(text):
    """부적합 키워드 체크"""
    found_keywords = []
    for keyword in INAPPROPRIATE_KEYWORDS:
        if keyword in text:
            found_keywords.append(keyword)
    return found_keywords

def check_special_chars(text):
    """특수문자/이모지 체크"""
    emoji_pattern = re.compile(
        "["
        "\U0001F600-\U0001F64F"
        "\U0001F300-\U0001F5FF"
        "\U0001F680-\U0001F6FF"
        "\U0001F1E0-\U0001F1FF"
        "]+", 
        flags=re.UNICODE
    )
    
    emojis = emoji_pattern.findall(text)
    return emojis

def validate_summary(result, strict=False):
    errors = []
    warnings = []
    
    # 1. 필수 키 존재 확인
    required_keys = ["shop_seq", "shop_name", "title", "summaries"]
    for key in required_keys:
        if key not in result:
            errors.append(f"❌ 필수 키 누락: {key}")
    
    if errors:
        return {"passed": False, "errors": errors, "warnings": warnings}
    
    # 2. 요약문 개수 확인
    summaries = result.get("summaries", [])
    if len(summaries) != 3:
        errors.append(f"❌ 요약문 개수 오류: {len(summaries)}개 (3개 필요)")
        return {"passed": False, "errors": errors, "warnings": warnings}
    
    # 3. 문장 길이 확인 (웨이팅 핫플은 조금 더 짧고 임팩트 있게)
    for i, sentence in enumerate(summaries, 1):
        length = len(sentence)
        if length < 30:
            warnings.append(f"⚠️ {i}번 문장이 너무 짧음: {length}자 (권장 35-55자)")
        elif length > 80:
            warnings.append(f"⚠️ {i}번 문장이 너무 김: {length}자 (권장 35-55자)")
    
    # 4. 제목 길이 확인
    title = result.get("title", "")
    title_length = len(title)
    if title_length < 15:
        warnings.append(f"⚠️ 제목이 너무 짧음: {title_length}자 (권장 20-40자)")
    elif title_length > 50:
        warnings.append(f"⚠️ 제목이 너무 김: {title_length}자 (권장 20-40자)")
    
    # 5. 예시 복사 의심 체크
    for i, sentence in enumerate(summaries, 1):
        similarity, similar_example = check_example_similarity(sentence)
        if similarity >= 0.9:
            errors.append(f"❌ {i}번 문장이 예시를 복사한 것으로 의심됨 (유사도 {similarity:.1%})")
        elif similarity >= 0.7:
            warnings.append(f"⚠️ {i}번 문장이 예시와 유사함 (유사도 {similarity:.1%})")
    
    # 6. 부적합 키워드 체크
    all_text = title + " ".join(summaries)
    inappropriate_found = check_inappropriate_keywords(all_text)
    if inappropriate_found:
        warnings.append(f"⚠️ 웨이팅 핫플에 부적합한 표현: {', '.join(inappropriate_found)}")
    
    # 7. 특수문자/이모지 체크
    emojis = check_special_chars(all_text)
    if emojis:
        warnings.append(f"⚠️ 이모지 발견: {', '.join(emojis)}")
    
    # 8. 문장 간 중복 체크
    for i in range(len(summaries)):
        for j in range(i+1, len(summaries)):
            similarity = calculate_similarity(summaries[i], summaries[j])
            if similarity >= 0.7:
                warnings.append(f"⚠️ {i+1}번과 {j+1}번 문장이 너무 유사함 (유사도 {similarity:.1%})")
    
    # 결과 판정
    if strict:
        passed = len(errors) == 0 and len(warnings) == 0
    else:
        passed = len(errors) == 0
    
    return {"passed": passed, "errors": errors, "warnings": warnings}

# 전체 결과 검증 실행
if len(all_results) == 0:
    print("⚠️ 검증할 결과가 없습니다.")
else:
    print("🔍 요약문 검증 시작\n")
    print("=" * 80)
    
    total_passed = 0
    total_warnings = 0
    total_failed = 0
    
    for idx, result in enumerate(all_results, 1):
        if result.get("status") == "error":
            continue
        
        shop_name = result.get("shop_name", "Unknown")
        print(f"\n[{idx}] {shop_name}")
        print("─" * 80)
        
        validation = validate_summary(result, strict=False)
        errors = validation["errors"]
        warnings = validation["warnings"]
        passed = validation["passed"]
        
        if passed and len(warnings) == 0:
            print("✅ 검증 통과: 모든 규칙을 준수합니다")
            total_passed += 1
        elif passed:
            print(f"✅ 검증 통과 (경고 {len(warnings)}개)")
            total_passed += 1
            total_warnings += 1
            for warning in warnings:
                print(f"   {warning}")
        else:
            print(f"❌ 검증 실패 (오류 {len(errors)}개)")
            total_failed += 1
            for error in errors:
                print(f"   {error}")
    
    print("\n" + "=" * 80)
    print("📊 검증 요약")
    print("=" * 80)
    print(f"✅ 통과: {total_passed}개")
    if total_warnings > 0:
        print(f"⚠️ 경고 있음: {total_warnings}개")
    if total_failed > 0:
        print(f"❌ 실패: {total_failed}개")

## ✅ 섹션 8: 결과 저장 (선택 사항)

### 저장 형식

#### 단일 매장 모드
- JSON 파일 1개
- 파일명: `summary_{매장명}_{타임스탬프}.json`

#### 다중 매장 모드
- JSON 파일 + CSV 파일
- 파일명: `summaries_batch_{타임스탬프}.json/csv`

In [ ]:
# ========================================
# 결과 저장
# ========================================

from datetime import datetime
import pandas as pd

if len(all_results) == 0:
    print("⚠️ 저장할 결과가 없습니다.")
else:
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    if MODE == "single":
        result_json = all_results[0]
        filename = f"summary_{result_json.get('shop_name')}_{timestamp}.json"
        
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(result_json, f, ensure_ascii=False, indent=2)
        
        print(f"✅ JSON 저장 완료: {filename}")
    
    elif MODE == "multi":
        # JSON 파일
        json_filename = f"summaries_batch_{timestamp}.json"
        with open(json_filename, 'w', encoding='utf-8') as f:
            json.dump(all_results, f, ensure_ascii=False, indent=2)
        
        print(f"✅ JSON 저장 완료: {json_filename}")
        
        # CSV 파일
        csv_data = []
        for result in all_results:
            if result.get("status") != "error":
                csv_data.append({
                    "shop_seq": result.get("shop_seq"),
                    "shop_name": result.get("shop_name"),
                    "title": result.get("title"),
                    "summary_1": result.get("summaries", [])[0] if len(result.get("summaries", [])) > 0 else "",
                    "summary_2": result.get("summaries", [])[1] if len(result.get("summaries", [])) > 1 else "",
                    "summary_3": result.get("summaries", [])[2] if len(result.get("summaries", [])) > 2 else "",
                    "status": "success"
                })
        
        df = pd.DataFrame(csv_data)
        csv_filename = f"summaries_batch_{timestamp}.csv"
        df.to_csv(csv_filename, index=False, encoding='utf-8-sig')
        
        print(f"✅ CSV 저장 완료: {csv_filename}")